In [3]:
import numpy as np

# ============================
# Vocabularios y mapeos básicos
# ============================
vocab_esp = ['yo', 'tú', 'él', 'ella', 'nosotros', 'ellos', 'como', 'bebo', 'veo', 'me gusta',
             'soy', 'eres', 'es', 'el', 'un', 'una', 'manzana', 'agua', 'libro', 'casa']

vocab_eng = ['i', 'you', 'he', 'she', 'we', 'they', 'eat', 'drink', 'see', 'like',
             'am', 'are', 'is', 'the', 'a', 'an', 'apple', 'water', 'book', 'house']

# Diccionarios para traducción directa
eng2esp = dict(zip(vocab_eng, vocab_esp))
esp2eng = dict(zip(vocab_esp, vocab_eng))

# Mapeos para embeddings y decodificación
esp2idx = {w:i for i,w in enumerate(vocab_esp)}
eng2idx = {w:i for i,w in enumerate(vocab_eng)}
idx2esp = {i:w for i,w in enumerate(vocab_esp)}
idx2eng = {i:w for i,w in enumerate(vocab_eng)}

# ============================
# Utilidades
# ============================

def softmax(x):
    e_x = np.exp(x - np.max(x, axis=-1, keepdims=True))
    return e_x / np.sum(e_x, axis=-1, keepdims=True)

def positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return angle_rads

def scaled_dot_product_attention(Q, K, V, mask=None):
    # Asegurarnos de que las dimensiones sean correctas para la transposición
    # Q, K, V deben tener forma (batch_size, seq_len, d_model)
    if len(Q.shape) == 3 and len(K.shape) == 3 and len(V.shape) == 3:
        # Formato esperado para entrada de transformer: (batch, seq, dim)
        matmul_qk = np.matmul(Q, K.transpose(0, 2, 1))
    else:
        # Adaptación para dimensiones menores o casos especiales
        Q_reshaped = Q.reshape(1, *Q.shape) if len(Q.shape) < 3 else Q
        K_reshaped = K.reshape(1, *K.shape) if len(K.shape) < 3 else K
        V_reshaped = V.reshape(1, *V.shape) if len(V.shape) < 3 else V

        # Asegurarnos de que tenga 3 dimensiones antes de transponer
        if len(K_reshaped.shape) == 3:
            matmul_qk = np.matmul(Q_reshaped, K_reshaped.transpose(0, 2, 1))
        else:
            # En último caso, última dimensión como profundidad
            matmul_qk = np.matmul(Q_reshaped, K_reshaped.T)

    dk = K.shape[-1]
    scaled_logits = matmul_qk / np.sqrt(dk)

    if mask is not None:
        scaled_logits += (mask * -1e9)

    attention_weights = softmax(scaled_logits)
    output = np.matmul(attention_weights, V)
    return output, attention_weights

def split_heads(x, num_heads):
    batch_size, seq_len, d_model = x.shape
    depth = d_model // num_heads
    x = x.reshape(batch_size, seq_len, num_heads, depth)
    x = x.transpose(0, 2, 1, 3)
    return x

def combine_heads(x):
    batch_size, num_heads, seq_len, depth = x.shape
    x = x.transpose(0, 2, 1, 3).reshape(batch_size, seq_len, num_heads*depth)
    return x

def multi_head_attention(Q, K, V, num_heads):
    batch_size = Q.shape[0] if len(Q.shape) >= 3 else 1
    seq_len = Q.shape[1] if len(Q.shape) >= 3 else Q.shape[0]
    d_model = Q.shape[-1]

    # Asegurarnos de que las entradas tienen la forma correcta (batch_size, seq_len, d_model)
    if len(Q.shape) < 3:
        Q = Q.reshape(1, *Q.shape)
    if len(K.shape) < 3:
        K = K.reshape(1, *K.shape)
    if len(V.shape) < 3:
        V = V.reshape(1, *V.shape)

    # Para casos con secuencias cortas, ajustar el número de cabezas
    safe_num_heads = min(num_heads, d_model // 4)  # Evitar divisiones con cero o muy pequeñas
    if safe_num_heads < 1:
        safe_num_heads = 1

    depth = d_model // safe_num_heads

    # Inicializar pesos con una semilla para consistencia
    np.random.seed(42)

    # Para casos simples o palabras individuales, usamos atención simple
    if seq_len == 1 or d_model < safe_num_heads * 2:
        output, _ = scaled_dot_product_attention(Q, K, V)
        return output

    # Intentar realizar la atención multi-cabeza normal para entradas más complejas
    try:
        # División en cabezas
        Q_split = []
        K_split = []
        V_split = []

        for i in range(safe_num_heads):
            start_idx = i * depth
            end_idx = (i + 1) * depth
            if end_idx > d_model:  # Evitar índices fuera de rango
                end_idx = d_model

            Q_split.append(Q[:, :, start_idx:end_idx])
            K_split.append(K[:, :, start_idx:end_idx])
            V_split.append(V[:, :, start_idx:end_idx])

        # Procesar cada cabeza por separado
        outputs = []
        for i in range(safe_num_heads):
            head_output, _ = scaled_dot_product_attention(Q_split[i], K_split[i], V_split[i])
            outputs.append(head_output)

        # Concatenar resultados si hay múltiples cabezas
        if len(outputs) > 1:
            concat = np.concatenate(outputs, axis=-1)
        else:
            concat = outputs[0]

        return concat

    except (ValueError, IndexError) as e:
        # Si falla la atención multi-cabeza, volver a la atención simple
        print(f"Usando atención simple debido a: {e}")
        output, _ = scaled_dot_product_attention(Q, K, V)
        return output

def layer_norm(x, eps=1e-6):
    mean = np.mean(x, axis=-1, keepdims=True)
    std = np.std(x, axis=-1, keepdims=True)
    return (x - mean) / (std + eps)

def feed_forward(x, d_ff):
    # Inicializar pesos con una semilla para reproducibilidad
    np.random.seed(42)
    W1 = np.random.randn(x.shape[-1], d_ff) * 0.1
    b1 = np.zeros(d_ff)
    W2 = np.random.randn(d_ff, x.shape[-1]) * 0.1
    b2 = np.zeros(x.shape[-1])

    x1 = np.maximum(0, np.dot(x, W1) + b1)
    x2 = np.dot(x1, W2) + b2
    return x2

def encoder_block(x, num_heads, d_ff):
    try:
        attn_output = multi_head_attention(x, x, x, num_heads)
        out1 = layer_norm(x + attn_output)
        ff_output = feed_forward(out1, d_ff)
        out2 = layer_norm(out1 + ff_output)
        return out2
    except Exception as e:
        print(f"Error en encoder_block: {e}")
        # Versión simplificada para casos problemáticos
        return x

def decoder_block(x, enc_output, num_heads, d_ff):
    try:
        masked_attn = multi_head_attention(x, x, x, num_heads)
        out1 = layer_norm(x + masked_attn)
        enc_dec_attn = multi_head_attention(out1, enc_output, enc_output, num_heads)
        out2 = layer_norm(out1 + enc_dec_attn)
        ff_output = feed_forward(out2, d_ff)
        out3 = layer_norm(out2 + ff_output)
        return out3
    except Exception as e:
        print(f"Error en decoder_block: {e}")
        # Versión simplificada para casos problemáticos
        return x

# ============================
# Funciones auxiliares para input/output
# ============================

def embed_sentence(sentence, vocab_dict, d_model):
    tokens = sentence.lower().split()
    embedded = np.zeros((len(tokens), d_model))
    for i, word in enumerate(tokens):
        idx = vocab_dict.get(word, 0)
        one_hot = np.zeros(len(vocab_dict))
        one_hot[idx] = 1
        embedded[i, :len(vocab_dict)] = one_hot
    return embedded[np.newaxis, :, :]

def decode_output(output, idx2word):
    tokens = []
    for vec in output[0]:
        idx = np.argmax(vec[:len(idx2word)])
        tokens.append(idx2word.get(idx, '?'))
    return ' '.join(tokens)

# ============================
# Función de traducción directa (palabra por palabra)
# ============================

def translate_simple(sentence):
    words = sentence.lower().split()
    translated = []
    for word in words:
        if word in eng2esp:
            translated.append(eng2esp[word])
        else:
            translated.append(f"[{word}]")  # palabra desconocida
    return ' '.join(translated)

# ============================
# Función principal traductor ing -> esp con modelo transformer
# ============================

def translate_eng2esp(sentence):
    d_model = 32  # Aumentado para mejor capacidad del modelo
    num_heads = 4
    d_ff = 64

    # Pre-procesamiento de la entrada
    words = sentence.lower().split()

    # Para evitar errores con entradas cortas
    if len(words) == 1:
        # Si solo hay una palabra, usar traducción directa
        if words[0] in eng2esp:
            print("Nota: Para una sola palabra, se usa traducción directa")
            return eng2esp[words[0]]

    # Continuar con el modelo de transformer para frases más largas
    try:
        x = embed_sentence(sentence, eng2idx, d_model)
        seq_len = x.shape[1]

        # Añadir codificación posicional
        pos_encoding = positional_encoding(seq_len, d_model)
        x_with_pos = x + pos_encoding[np.newaxis, :seq_len, :]

        # Aplicar bloques de transformer
        enc_out = encoder_block(x_with_pos, num_heads, d_ff)
        dec_out = decoder_block(x_with_pos, enc_out, num_heads, d_ff)

        # Decodificar la salida
        translated = decode_output(dec_out, idx2esp)
        return translated

    except Exception as e:
        print(f"Error en el modelo transformer: {e}")
        print("Usando traducción de respaldo palabra por palabra...")
        return translate_simple(sentence)

# ============================
# Interfaz simple
# ============================

def run_simple_translator():
    print("Traductor básico ING->ESP (palabra por palabra)")
    print("Vocabulario ING permitido:", ', '.join(vocab_eng))

    while True:
        inp = input("\nEscribe una frase en inglés (o 'salir'): ").strip().lower()
        if inp == "salir":
            break
        output = translate_simple(inp)
        print("Entrada:", inp)
        print("Traducción palabra por palabra:", output)

def run_transformer_translator():
    print("Traductor ING->ESP basado en Transformer")
    print("Vocabulario ING permitido:", ', '.join(vocab_eng))
    print("Nota: Para mejores resultados, usa frases completas.")

    while True:
        inp = input("\nEscribe una frase en inglés (o 'salir'): ").strip().lower()
        if inp == "salir":
            break

        words = inp.split()
        unknown_words = [word for word in words if word not in eng2idx]

        if unknown_words:
            print(f"❌ Palabras desconocidas: {', '.join(unknown_words)}")
            print("Solo se permiten palabras del vocabulario definido.")
        else:
            print("Procesando...")
            try:
                output = translate_eng2esp(inp)
                print("Entrada:", inp)
                print("Traducción con transformer:", output)

                # Mostrar también la traducción palabra por palabra para comparar
                simple = translate_simple(inp)
                if output != simple:
                    print("Traducción palabra por palabra:", simple)
            except Exception as e:
                print(f"Error al traducir: {str(e)}")
                print("Usando traducción simple como respaldo:")
                print(translate_simple(inp))

if __name__ == "__main__":
    print("Selecciona el tipo de traductor:")
    print("1: Traductor básico (palabra por palabra)")
    print("2: Traductor avanzado (con modelo transformer)")

    choice = input("Ingresa tu opción (1 o 2): ")

    if choice == "1":
        run_simple_translator()
    elif choice == "2":
        run_transformer_translator()
    else:
        print("Opción inválida. Ejecutando traductor básico por defecto.")
        run_simple_translator()

Selecciona el tipo de traductor:
1: Traductor básico (palabra por palabra)
2: Traductor avanzado (con modelo transformer)
Ingresa tu opción (1 o 2): 2
Traductor ING->ESP basado en Transformer
Vocabulario ING permitido: i, you, he, she, we, they, eat, drink, see, like, am, are, is, the, a, an, apple, water, book, house
Nota: Para mejores resultados, usa frases completas.

Escribe una frase en inglés (o 'salir'): you
Procesando...
Nota: Para una sola palabra, se usa traducción directa
Entrada: you
Traducción con transformer: tú

Escribe una frase en inglés (o 'salir'): 1
❌ Palabras desconocidas: 1
Solo se permiten palabras del vocabulario definido.

Escribe una frase en inglés (o 'salir'): salir
